In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
import matplotlib.style as style
style.use('ggplot') #just changing seaborn style
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.metrics import precision_score, recall_score, \
confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from random import randint
import random
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
import category_encoders as encoders
from sklearn.ensemble import VotingClassifier
from sklearn import linear_model
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import mean_squared_error
from scipy.sparse import coo_matrix

In [3]:
class DataChanger(BaseEstimator, TransformerMixin):
    """Replacing nan values for loan prediction data set"""
    
    def __init__(self):
        pass
        
      
    def fit(self, data, y=None):
        """mandatory method for transformer"""        
        return self
    

    """_______________________________| METHOD FOR WEIGHTS NAN VALUES |________________________________"""
    
    
    def weights_nan(self, data):
        """fillna weight values - lf same products and fill weight based on that"""
        if "Item_Weight" in data.columns: #if column was not deleted
            
            weights_and_id = {}
            
            mart_for_weights = data.dropna(subset=["Item_Weight"]) #droping nan values
            
            for identifier, weight in zip(mart_for_weights["Item_Identifier"],mart_for_weights["Item_Weight"]):
                weights_and_id[identifier] = weight
            #paraller iterating through columns and creating a dictionary with id a weights
            
            data.set_index("Item_Identifier", inplace=True) #seting index for identifier

            data["Item_Weight"].update(pd.Series(weights_and_id)) 
            #updating nan values from dictionary
            data.reset_index(inplace=True) #reseting index back to numeric values
            
            #Replace the rest of values
            data["Item_Weight"].fillna(data['Item_Weight'].median(), inplace=True)
            
            
    
    """_________________________| METHOD FOR REPAIRING FAT_CONTENT TYPOS |____________________________"""
            
    def fat_content(self, data):
        """just replacing typos in fat content category and rename values"""
        if "Item_Fat_Content" in data.columns: #if column isn't deleted
            data["Item_Fat_Content"][(data["Item_Fat_Content"]==  "LF")] = "Low Fat"
            data["Item_Fat_Content"][(data["Item_Fat_Content"]==  "low fat")] = "Low Fat"
            data["Item_Fat_Content"][(data["Item_Fat_Content"]==  "reg")] = "Regular"
            #just replacing typos nothing else 
        
        
    def outlet_size_nan(self, data):
        if "Outlet_Size" in data.columns:
            data["Outlet_Size"][(data["Outlet_Identifier"] =="OUT010")] = "Small"
            data["Outlet_Size"][(data["Outlet_Identifier"] =="OUT017")] = "Medium"
            data["Outlet_Size"][(data["Outlet_Identifier"] =="OUT045")] = "High"
        #Its not generic at all - only works for this column..
        
    
    
    """____________________________| METHOD FOR REPAIRING VISIBILITY (0) |_______________________________"""
    
    def visibility(self, data):
        """fill visbility value based on same product but in different stores -> only for 
        grocery type I has to use coefficient +/- *1,7 because of lower number of products"""
        if "Item_Visibility" in data.columns: #if column isn't deleted
            
            """CREATING NECESSARY LISTS/SERIES - data to filled in"""
            
            #i am creating series from visiblity products and their means for all type of
            #outlet except grocery stores
            visibility_normal = data[(data["Item_Visibility"] != 0) & \
                                        (data["Outlet_Type"] != "Grocery Store")]\
            .groupby("Item_Identifier").mean()["Item_Visibility"]
            #i wanna non zero values - no mean bias
            
            #same thing but only for grocery stores - they have smaller number of products
            #and therefore different visibility
            visibility_grocery = data[(data["Item_Visibility"] != 0) &\
                                         (data["Outlet_Type"] == "Grocery Store")]\
            .groupby("Item_Identifier").mean()["Item_Visibility"]
            
            #I calculate second series for grocery because in visibility grocery is a lot of them 
            #missing - so I am using simple ratio * product visibility in regular outlet
            visibility_normal_grocery= visibility_normal*(visibility_grocery.mean()/visibility_normal.mean())
            #ration is something about 1,7 but i calculcate more precisely this way
            
            
            """FILLED IN DATA ABOVE AND GET RID OF ZERO VALUES IN VISIBILITY COLUMN"""
            
            #firstly i overwrite zero values with nan but only for NON grocery stores
            data["Item_Visibility"][(data["Item_Visibility"] == 0) & \
                                       (data["Outlet_Type"] != "Grocery Store")] = np.nan
                
            data.set_index("Item_Identifier", inplace = True) # i need to set index so it matches
            #with my pd.series
            data.update(visibility_normal, overwrite=False) #then i update valueswith visibility normal
            #series - overwrite is False so it is updating only nan values (my zeros in Grocery are left
            #unchanged) - no need to specify column - It is using pd.series name as a column name

            #filling nan but now for grocery stores
            data["Item_Visibility"][(data["Item_Visibility"] == 0) & \
                                       (data["Outlet_Type"] == "Grocery Store")] = np.nan
            data.update(visibility_grocery, overwrite=False) #replacing nan values
            data.update(visibility_normal_grocery, overwrite=False) #and again but with second
            #version of my data - I am not doing this in first case because it's a larger set no 
            #added value
            data.reset_index(inplace=True)
            
            """FILLED IN LAST REMAINIGN ZEROS WITH MEAN VALUES"""
            #NOTE: there are not any zeros left but this is a good insurance
            if data["Item_Visibility"].isna().any(): #don't proceed if there are no nan values
                #calculate mean for non grocery and grocery stores
                mean_non_grocery = data["Item_Visibility"][(data["Outlet_Type"] != "Grocery Store")].mean()
                mean_grocery =  data["Item_Visibility"][(data["Outlet_Type"] == "Grocery Store")].mean()
            
                #fillna for non grocery
                data.loc[(data["Outlet_Type"] != "Grocery Store"), "Item_Visibility"] = \
                data.loc[(data["Outlet_Type"] != "Grocery Store"), "Item_Visibility"].fillna(mean_non_grocery)
                
                #and for grocery
                data.loc[(data["Outlet_Type"] == "Grocery Store"), "Item_Visibility"] = \
                data.loc[(data["Outlet_Type"] == "Grocery Store"), "Item_Visibility"].fillna(mean_grocery)
                
                   
    """_____________________________________________| SUMMARY|__________________________________________"""
    
    def transform(self, data):
        self.weights_nan(data) #fill nan values for item_weight column
        self.fat_content(data) #repair typos in item_fat_content column
        self.visibility(data) #fill zero values in visibility column
        self.outlet_size_nan(data)
        
        
        return data #return final table - after fit_transform or transform 
    

In [322]:
class NumCatChanger(BaseEstimator, TransformerMixin):
    """Transformers for categorical and numerical data"""
    
    def __init__(self, 
                 transformer_name = StandardScaler, #default value 
                 column_transfer = [], #changes columns from nominal to ordinal
                 num_ordina_scaler = StandardScaler, #second optinal scaling
                 switch_pipe = False, #switch from ordinal to ordinal and scaling
                ):
        
        self.transformer_name = transformer_name
        self.column_transfer = column_transfer
        self.mappings = []
        self.num_ordina_scaler  = num_ordina_scaler
        self.switch_pipe = switch_pipe
        
        
    def fit(self, data, y=None):
        """mandatory method for transformer"""        
        return self  
        
        
    def transform(self, data):
        """Set of transformers"""
        numeric_columns = [col for col in data.columns if "_NUM" in col] #columns for numeric pipelines
        #all of my columns are supposed to be nominal, if I don't say otherwise 
        #it's a list of all columns having "CAT in name and are not in my self.column_transfer list"
        nominal_columns = [col for col in data.columns if "_CAT" in col if col not in self.column_transfer]
        ordinal_columns = self.column_transfer #optional ordinal columns - based on my transformer parameter

        #Mapping for ordinal if necessary - in this case only out of curiosity
        #I wont use ordinal for Item ID and Outer ID and Location and Fat_Content
        Outlet_Type_ORD = {'col': 'Outlet_Type_CAT', 'mapping': {"Grocery Store" : 1, "Supermarket Type1": 2, 
                                                                 "Supermarket Type2": 3, "Supermarket Type3" : 4}}
        
        Outlet_Size_ORD = {'col': 'Outlet_Size_CAT', 'mapping': {"Small" : 1, "Medium" : 2, "High" : 3}}
        Outlet_Year_ORD = {'col': 'Outlet_Establishment_Year_CAT', 'mapping': {"new" : 1, 
                                                                               "old" : 2,
                                                                               }}
                           
        #Activate mapping 
        if "Outlet_Type_CAT" in self.column_transfer:
            self.mappings.append(Outlet_Type_ORD)
        if "Outlet_Size_CAT" in self.column_transfer:
            self.mappings.append(Outlet_Size_ORD)
        if "Outlet_Establishment_Year_CAT" in self.column_transfer: #debug
            self.mappings.append(Outlet_Year_ORD)
        
        #individual pipelines
        #For numerical values - I can pick what scaler i want in pipe parameters
        num_pipeline = Pipeline([("optional num transformer", self.transformer_name())]) #regular
        num_pipeline2 = Pipeline([("optional num transformer", self.num_ordina_scaler())])#for num_ord_col
        #I am having two pipes - one for regular numbers and second for case when I am also scaling ordinal
        
        #For nominal values - this one is easy just 1 and 0 
        nominal_pipeline = Pipeline([("One_Hot_Encoder", OneHotEncoder())])
        #For ordinal values - uses my mapping from above!
        
        #Now i generate True and False values for my Pipes - depends if I am using a switcher or not
        if not self.switch_pipe:
            ordinal_pipeline = Pipeline([("Ordinal_Encoder", encoders.OrdinalEncoder(mapping = self.mappings))])
            #If my switcher is False - I am using only ordinal 
        else:
            ordinal_pipeline = Pipeline([("Ordinal_Encoder", encoders.OrdinalEncoder(mapping = self.mappings))])
            nom_ord_pipe = Pipeline(steps=[("ordinal", ordinal_pipeline), ("ordi_num", num_pipeline2)])
            ordinal_pipeline = False
            #If my switcher is True I am using ordinal - and immediately after that scaler of my choice
        
        
        #final pipeline
        colum_transformer = ColumnTransformer([("numeric", num_pipeline, numeric_columns),
                                               ("nominal", nominal_pipeline, nominal_columns),
                                               ("ordinal",ordinal_pipeline or nom_ord_pipe, ordinal_columns),
                                               ],remainder='drop') #passthrough
        return colum_transformer.fit_transform(data)      
    
    #and = False or END, or True or END

In [5]:
class OptionalChanger(BaseEstimator, TransformerMixin):
    """Change data"""
    
    def __init__(self, 
                columns_to_drop = None, #drops columns
                change_year = True, #changes year to old and new
                year_new = 2008, #year from which is store evaluated as a new one
                change_outo18 = False,#changes outo18 Outlet_Type
                change_size = False, #changes sizes of stores
                rescale_visibility = False,
                ): 
        
        self.columns_to_drop = columns_to_drop
        self.change_year = change_year
        self.year_new = year_new
        self.change_outo18 = change_outo18
        self.change_size = change_size
        self.rescale_visibility = rescale_visibility
        
        
        
    def fit(self, data, y=None):
        """mandatory method for transformer"""        
        return self
    

    """_______________________________| YEAR ESTABLISHMENT - CHANGE |_______________________________"""
    
    
    def year_change(self, data):
        """Changes years only to old and new"""
        if self.change_year:
            data["Outlet_Establishment_Year"]\
            [(data["Outlet_Establishment_Year"]>self.year_new)] = "new"
            data["Outlet_Establishment_Year"]\
            [(data["Outlet_Establishment_Year"] != "new")] = "old"
        #I can also evaluate as 1 and 0 ...
        
        
    """__________________________________| OUTO18 TYPE  - CHANGE |___________________________________"""
    
    def outo18_type(self,data):
        """Changes type of store for outo18"""
        if self.change_outo18:
            data["Outlet_Type"][(data["Outlet_Identifier"]== "OUT018")] = "Supermarket Type1"
            #this will mainly affect encoding for outlet type - outo18 is only one in type2 category
            #and is very similar to others in type2 
                   
    
    """___________________________________| OUTLET_SIZE - CHANGE |____________________________________"""
    
    
    def change_outlet_size(self,data):
        if self.change_size:
            data["Outlet_Size"][(data["Outlet_Type"] == "Grocery Store")] = "Small"
            data["Outlet_Size"][(data["Outlet_Type"] != "Grocery Store")] = "Big"
        #much simplier method - and can be also used for nan values
        
   
    """___________________________________| VISIBLITY - CHANGE |____________________________________"""
    
    #NOTE: rescaler would take care of it probably, but it is fun task to do :)
    
    def visibility_rescale(self, data):
        """changes the visibility from 60 to 100"""
        if self.rescale_visibility: 
            #firstly I am creating list of stores in my table
            outlet_list = list(data["Outlet_Identifier"].value_counts().index)
            #counting sum of visibility for each store
            sum_visibility = data[["Item_Visibility","Outlet_Identifier"]].groupby("Outlet_Identifier").sum()
            #and calculate their ratio to 100%
            sum_visibility["coef"] = (100/sum_visibility["Item_Visibility"])
            #creationg a list with ration
            coef_list = list(sum_visibility["coef"])
            #my result is something like - OUTO49, ratio 1.5 --> meaning each value has to be 
            #multiplicated *1,5 -> to achieve that final sum of visibility values is 100 (for each store)
            
            #now i will iterate through all stores and change their visiblity values based on their coef
            for i in range(len(outlet_list)):
                #this part filter table based on store ID and indicates that we want change item visibility
                data.loc[(data["Outlet_Identifier"] == outlet_list[i]), "Item_Visibility"] = \
                data["Item_Visibility"][(data["Outlet_Identifier"] == outlet_list[i])]*coef_list[i]
                #this part filter the values I want change and multiplicate them by its coefficient
    
    
    """_____________________________________| COLUMNS - RENAMING |______________________________________"""
    
    def rename_columns(self, data):
        """Renaming function - if columns are not in data they won't be affected""" 
        new_names = {"Item_Identifier": "Item_Identifier_CAT", 
                     "Item_Fat_Content": "Item_Fat_Content_CAT",
                     "Item_Type" : "Item_Type_CAT",
                    "Outlet_Identifier": "Outlet_Identifier_CAT",
                    "Outlet_Establishment_Year": "Outlet_Establishment_Year_CAT",
                    "Outlet_Size" : "Outlet_Size_CAT",
                    "Outlet_Location_Type" : "Outlet_Location_Type_CAT",
                    "Outlet_Type" : "Outlet_Type_CAT",
                    
                    "Item_Weight" : "Item_Weight_NUM" ,
                    "Item_Visibility" : "Item_Visibility_NUM",
                    "Item_MRP" : "Item_MRP_NUM",}
        return data.rename(columns=new_names, inplace=True)

    """_____________________________________________| SUMMARY|__________________________________________"""

    def transform(self, data):
        
        self.year_change(data) #changes years to old and new
        self.outo18_type(data) #changes type for outo18 from type2 to type1
        self.change_outlet_size(data) #changes size of stores
        self.visibility_rescale(data)
        
        
        #drops optional number of columns
        self.columns_to_drop and data.drop(self.columns_to_drop, 1, inplace=True)#short circuit with and
        
        
        self.rename_columns(data) #renaminig my columns
    
            
        return data

In [89]:
class ColumnsAdder(BaseEstimator, TransformerMixin):
    "Just adding some columns"
    def __init__(self,
                 item_std = False,
                 outlet_item_fat_std = False,
                 mean_values = False):
        
        self.item_std = item_std
        self.outlet_item_fat_std = outlet_item_fat_std
        self.mean_values = mean_values
        
    def fit(self, data, y=None):
        return self
        

    def transform(self, data):
        
        """Done"""
        def fitem_std(row): #grouping based only on Item ID
            for item in set(data["Item_Identifier"]):
                if row["Item_Identifier"] == item:
                    value = data.groupby("Item_Identifier").std()["Item_MRP"][item]
            return value
          
        """Testing""" 
        def foutlet_item_fat_std(row): #grouping based on outlet - item typ and fat content
            for item in set(data["Outlet_Identifier"]):
                for item2 in set(data["Item_Type"]):
                    for item3 in set(data["Item_Fat_Content"]):
                        if row["Outlet_Identifier"] == item and row["Item_Type"] \
                        == item2 and row["Item_Fat_Content"] == item3:
                            value = data.groupby(["Outlet_Identifier","Item_Type","Item_Fat_Content"]).std()\
                            ["Item_MRP"][item][item2][item3]
            return value
      
        """Testing"""
        def fmean_values(row): #only mean grouping
            for item in set(data["Outlet_Identifier"]):
                for item2 in set(data["Item_Type"]):
                    for item3 in set(data["Item_Fat_Content"]):
                        if row["Outlet_Identifier"] == item and row["Item_Type"] \
                        == item2 and row["Item_Fat_Content"] == item3:
                            value = data.groupby(["Outlet_Identifier","Item_Type","Item_Fat_Content"]).mean()\
                            ["Item_MRP"][item][item2][item3]
            return value
        
        
        """Calling my functions"""
        if self.item_std:
            data.insert(2,"item_std_NUM", data.apply(fitem_std, axis=1),True)
            data.fillna(data["item_std_NUM"].mean(),inplace=True)
            
        if self.outlet_item_fat_std:
            data.insert(2,"outlet_item_fat_std_NUM",data.apply(foutlet_item_fat_std, axis=1),True)
            data.fillna(data["outlet_item_fat_std_NUM"].mean(),inplace=True)
            
        if self.mean_values:
            data.insert(2,"mean_values_NUM", data.apply(fmean_values, axis=1),True)
            data.fillna(data["mean_values_NUM"].mean(),inplace=True)


        return data
            
        

In [54]:
my_data = DataChanger().fit_transform(pd.read_csv("train_big_mart.csv"))
my_data = OptionalChanger(columns_to_drop = "Item_Identifier", 
                          change_year=True,
                          change_outo18=True,
                          change_size=False,
                          rescale_visibility=False).fit_transform(my_data)

In [55]:
def triple_data():
    data_triple = pd.read_csv("train_big_mart.csv") 
    data_triple["Item_Fat_Content"][(data_triple["Item_Fat_Content"]==  "LF")] = "Low Fat"
    data_triple["Item_Fat_Content"][(data_triple["Item_Fat_Content"]==  "low fat")] = "Low Fat"
    data_triple["Item_Fat_Content"][(data_triple["Item_Fat_Content"]==  "reg")] = "Regular"
    return data_triple

In [73]:
pokus = triple_data() 

In [78]:
pokus.groupby(["Outlet_Identifier","Item_Type","Item_Fat_Content"]).mean()

Item_Weight  \
Outlet_Identifier Item_Type     Item_Fat_Content                
OUT010            Baking Goods  Low Fat             12.922500   
                                Regular             12.317955   
                  Breads        Low Fat             13.784286   
                                Regular             11.934500   
                  Breakfast     Low Fat             11.032500   
...                                                       ...   
OUT049            Snack Foods   Regular             12.234052   
                  Soft Drinks   Low Fat             11.873378   
                                Regular             11.826667   
                  Starchy Foods Low Fat             14.933333   
                                Regular             12.931875   

                                                  Item_Visibility    Item_MRP  \
Outlet_Identifier Item_Type     Item_Fat_Content                                
OUT010            Baking Goods  Low Fat                  0.126432  123.639060   
                                Regular                  0.131011  129.803136   
                  Breads        Low Fat                  0.082788  170.939914   
                                Regular                  0.110149  143.639380   
                  Breakfast     Low Fat                  0.103027  182.670000   
...                                                           ...         ...   
OUT049            Snack Foods   Regular                  0.062857  142.349748   
                  Soft Drinks   Low Fat                  0.061019  126.023541   
                                Regular                  0.062304  119.036689   
                  Starchy Foods Low Fat                  0.066803  151.599178   
                                Regular                  0.064668  147.033575   

                                                  Outlet_Establishment_Year  \
Outlet_Identifier Item_Type     Item_Fat_Content                              
OUT010            Baking Goods  Low Fat                                1998   
                                Regular                                1998   
                  Breads        Low Fat                                1998   
                                Regular                                1998   
                  Breakfast     Low Fat                                1998   
...                                                                     ...   
OUT049            Snack Foods   Regular                                1999   
                  Soft Drinks   Low Fat                                1999   
                                Regular                                1999   
                  Starchy Foods Low Fat                                1999   
                                Regular                                1999   

                                                  Item_Outlet_Sales  
Outlet_Identifier Item_Type     Item_Fat_Content                     
OUT010            Baking Goods  Low Fat                  247.411280  
                                Regular                  261.144918  
                  Breads        Low Fat                  549.665457  
                                Regular                  380.970760  
                  Breakfast     Low Fat                  443.922150  
...                                                             ...  
OUT049            Snack Foods   Regular                 2332.354797  
                  Soft Drinks   Low Fat                 2202.268459  
                                Regular                 1789.078578  
                  Starchy Foods Low Fat                 2439.713133  
                                Regular                 2507.818925  

[279 rows x 5 columns]

In [88]:
ColumnsAdder(mean_values=True).fit_transform(triple_data()).info() #TOHLE SMAZAT 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   mean_values_NUM            8523 non-null   float64
 3   Item_Fat_Content           8523 non-null   object 
 4   Item_Visibility            8523 non-null   float64
 5   Item_Type                  8523 non-null   object 
 6   Item_MRP                   8523 non-null   float64
 7   Outlet_Identifier          8523 non-null   object 
 8   Outlet_Establishment_Year  8523 non-null   int64  
 9   Outlet_Size                6113 non-null   object 
 10  Outlet_Location_Type       8523 non-null   object 
 11  Outlet_Type                8523 non-null   object 
 12  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(5), int64(1), object(7)
memory usage:

In [90]:
ColumnsAdder(outlet_item_fat_std=True).fit_transform(triple_data()) #TOHLE SPUSTIT

,Item_Identifier,Item_Weight,outlet_item_fat_std_NUM,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,76.022148,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,56.728905,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,52.756353,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,58.114699,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,61.6944,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,65.792439,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8518,FDF22,6.865,55.269360,Low Fat,0.056783,Snack Foods,214.5218,OUT013,1987,High,Tier 3,Supermarket Type1,2778.3834
8519,FDS36,8.380,61.099251,Regular,0.046982,Baking Goods,108.1570,OUT045,2002,61.6944,Tier 2,Supermarket Type1,549.2850
8520,NCJ29,10.600,57.181155,Low Fat,0.035186,Health and Hygiene,85.1224,OUT035,2004,Small,Tier 2,Supermarket Type1,1193.1136
8521,FDN46,7.210,60.714932,Regular,0.145221,Snack Foods,103.1332,OUT018,2009,Medium,Tier 3,Supermarket Type2,1845.5976


In [93]:
rmse = [1,2,3,4,5,6,7,8,9,10]

In [94]:
rmse = np.array(rmse).reshape(5,2)

In [95]:
rmse

array([[ 1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 7,  8],
       [ 9, 10]])

# SUPER OPERACE!! :)

In [ ]:
trying = X_train.copy()

In [ ]:
trying = DataChanger().fit_transform(trying)

In [ ]:
trying["prerank"] = trying["Item_Weight"]*trying["Item_Visibility"] #prerank weights * visibility

In [ ]:
trying["rank"] = trying.groupby(["Outlet_Identifier","Item_Type"])["prerank"].rank("first") #ranking

In [ ]:
trying['rank_sum'] = trying.groupby(["Outlet_Identifier","Item_Type"])['rank'].transform(
                           lambda x: x.count()) #suming ranks for each group - OUTLET x TYPE OF GOODS

In [ ]:
#trying[(trying["Outlet_Identifier"] == "OUT035")& (trying["Item_Type"]=="Breads")] #just useful filter

In [ ]:
trying["percentil"] = trying["rank"] / trying["rank_sum"]

In [ ]:
x = np.array(trying['percentil'].values.tolist())
trying['percentil'] = np.where(x < 0.5, (1.5-x) ,1-(x-0.5)).tolist()

In [ ]:
trying["MRP_OP"] = trying["Item_MRP"]*trying["percentil"]

# TAKHLE JSEM SKVĚLE VYDOLOVAL PERCENTILY